## Add channel files based on the EEG with fixes.

This notebook replaces the `channels.tsv` files of ESS with those generated by EEGLAB BidsTools
for the BCIT Baseline Driving data.


In [1]:
import os
import json
import datetime
import pandas as pd
from hed.tools import HedLogger
from hed.util import get_file_list, get_value_dict

# Variables to set for the specific dataset
bids_root_path = '/XXX/bcit/BaselineDrivingWorkingPhaseTwo'
exclude_dirs = ['sourcedata', 'stimuli', 'code']
log_name = 'bcit_baseline_driving_07_fix_channels_log'
sampling_rate_file = os.path.realpath(os.path.join(bids_root_path, 'code/samplingRates.tsv'))
channel_file = os.path.realpath(os.path.join(bids_root_path, "code/channelsOriginal.json"))

# Set up the logger
log_file_name = f"code/curation_logs/{log_name}.txt"
logger = HedLogger(name=log_name)

# Load the channels file
with open(channel_file, 'r') as fp:
    channel_info = json.load(fp)

# Construct the lists of the EEG files
eeg_files = get_file_list(bids_root_path, extensions=[".set"], name_suffix="_eeg", exclude_dirs=exclude_dirs)

# Get the sample rate dictionary
sampling_dict = get_value_dict(sampling_rate_file)

chan_dict = {'LN': ('LN', 'OTHER', 'n/a', 'Lane deviation from center line in meters.'),
             'ANG': ('ANG', 'OTHER', 'n/a', 'Steering wheel deviation angle in degrees.'),
             'SP': ('SP', 'OTHER', 'n/a', 'Vehicle speed in mph.'),
             'SD': ('SD', 'OTHER', 'n/a', 'Vehicle speed deviation.'),
             'EXG1': ('LHEOG', 'EOG', 'uV', 'Left horizontal EOG'),
             'EXG2': ('RHEOG', 'EOG', 'uV', 'Right horizontal EOG'),
             'EXG3': ('UVEOG', 'EOG', 'uV', 'Upper vertical EOG'),
             'EXG4': ('LVEOG', 'EOG', 'uV', 'Lower vertical EOG'),
             'EXG5': ('LMAST', 'MISC', 'uV', 'Left mastoid'),
             'EXG6': ('RMAST', 'MISC', 'uV', 'Right mastoid')}

for file_name in eeg_files:
    basename = os.path.basename(file_name)
    channels = channel_info[basename]
    logger.add(basename, f"EEG has {len(channels)} channels")
    sampling_key = basename[:-8]
    sampling_rate = sampling_dict[sampling_key]
    logger.add(basename, f"Sampling rate is {sampling_rate}")
    data = {'name': channels}
    df = pd.DataFrame(data)
    df[['type', 'units', 'sampling_frequency', 'description']] = 'n/a'
    names = df['name']
    for index, name in names.iteritems():
        row = chan_dict.get(name, None)
        if not row:
            row = (name, 'EEG', 'uV', 'n/a')
        df.loc[index, 'name'] = row[0]
        df.loc[index, 'type'] = row[1]
        df.loc[index, 'units'] = row[2]
        df.loc[index, 'sampling_frequency'] = sampling_rate
        df.loc[index, 'description'] = row[3]
    EEG = sum(df['type'].map(str) == 'EEG')
    EOG = sum(df['type'].map(str) == 'EOG')
    OTHER = sum(df['type'].map(str) == 'OTHER')
    MISC = sum(df['type'].map(str) == 'MISC')
    logger.add(basename, f"{EEG} EEG channels, {EOG} EOG channels, {OTHER} OTHER channels {MISC} MISC channels")
    file_new = file_name[:-7] + "channels.tsv"
    df.to_csv(file_new, sep='\t', index=False)
    logger.add(basename, f"Saved as {os.path.basename(file_new)}")

# Output and save the log
log_string = "\n\nLog output:\n" + logger.get_log_string()
error_string = "\n\nERROR Summary:\n" + logger.get_log_string(level="ERROR")
print(log_string)
print(error_string)

save_path = os.path.join(bids_root_path, log_file_name)
with open(save_path, "w") as fp:
    fp.write(f"{log_file_name} {datetime.datetime.now()}\n")
    fp.write(log_string)
    fp.write(error_string)



Log output:
bcit_baseline_driving_07_fix_channels_log: Level None
sub-01_ses-01_task-DriveWithSpeedChange_run-1_eeg.set:
	[ EEG has 74 channels]
	[ Sampling rate is 2048]
	[ 64 EEG channels, 4 EOG channels, 4 OTHER channels 2 MISC channels]
	[ Saved as sub-01_ses-01_task-DriveWithSpeedChange_run-1_channels.tsv]
sub-02_ses-01_task-DriveWithSpeedChange_run-1_eeg.set:
	[ EEG has 74 channels]
	[ Sampling rate is 2048]
	[ 64 EEG channels, 4 EOG channels, 4 OTHER channels 2 MISC channels]
	[ Saved as sub-02_ses-01_task-DriveWithSpeedChange_run-1_channels.tsv]
sub-03_ses-01_task-DriveWithSpeedChange_run-1_eeg.set:
	[ EEG has 74 channels]
	[ Sampling rate is 2048]
	[ 64 EEG channels, 4 EOG channels, 4 OTHER channels 2 MISC channels]
	[ Saved as sub-03_ses-01_task-DriveWithSpeedChange_run-1_channels.tsv]
sub-04_ses-01_task-DriveWithSpeedChange_run-1_eeg.set:
	[ EEG has 74 channels]
	[ Sampling rate is 2048]
	[ 64 EEG channels, 4 EOG channels, 4 OTHER channels 2 MISC channels]
	[ Saved as sub-